In [923]:
!pip install dmba

In [924]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import dmba
from dmba import classificationSummary
from sklearn.linear_model import LogisticRegression

In [925]:
#Load files
ml = pd.read_csv('/Users/ianmabon/Desktop/Last Semester!/Sports/NBA Betting/nba_betting_money_line.csv')
spread = pd.read_csv('/Users/ianmabon/Desktop/Last Semester!/Sports/NBA Betting/nba_betting_spread.csv')
totals = pd.read_csv('/Users/ianmabon/Desktop/Last Semester!/Sports/NBA Betting/nba_betting_totals.csv')
games = pd.read_csv('/Users/ianmabon/Desktop/Last Semester!/Sports/NBA Betting/nba_games_all.csv')
teams = pd.read_csv('/Users/ianmabon/Desktop/Last Semester!/Sports/NBA Betting/nba_teams_all.csv')
games_complete = pd.read_csv('/Users/ianmabon/Desktop/Last Semester!/Sports/NBA Betting/Kaggle/game.csv')
print(ml.shape)
print(spread.shape)
print(totals.shape)
print(games.shape)
print(teams.shape)
print(games_complete.shape)

(125286, 7)
(131690, 9)
(131386, 9)
(125624, 32)
(69, 5)
(65698, 55)


In [926]:
#Merge datasets and group by book
books = {book: group for book, group in ml.groupby('book_name')}
games = games.merge(
    games_complete[['game_id', 'pts_away', 'pts_home', 'team_id_home', 'team_id_away']],
    on='game_id',
    how='left'
)
games.rename(columns={'a_team_id': 'game_a_team_id', 'team_id': 'game_team_id'}, inplace=True)

merged_books = {}
#Merge for each book
for book, book_df in books.items():
    merge1 = pd.merge(book_df, spread, on=['game_id', 'book_id'], how='inner')
    merge2 = pd.merge(merge1, games, on='game_id', how='inner')
    merge3 = pd.merge(merge2, totals, on='game_id', how='inner')
    merge3 = merge3.drop_duplicates(subset='game_id')
    merged_books[book] = merge3

#Sanity check
print(merged_books['YouWager'].columns)
merged_books['5Dimes'].head()

Index(['game_id', 'book_name_x', 'book_id_x', 'team_id_x', 'a_team_id_x',
       'price1_x', 'price2_x', 'book_name_y', 'team_id_y', 'a_team_id_y',
       'spread1', 'spread2', 'price1_y', 'price2_y', 'game_date', 'matchup',
       'game_team_id', 'is_home', 'wl', 'w', 'l', 'w_pct', 'min', 'fgm', 'fga',
       'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb',
       'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'game_a_team_id', 'season_year', 'season_type', 'season', 'pts_away',
       'pts_home', 'team_id_home', 'team_id_away', 'book_name', 'book_id_y',
       'team_id', 'a_team_id', 'total1', 'total2', 'price1', 'price2'],
      dtype='object')


,game_id,book_name_x,book_id_x,team_id_x,a_team_id_x,price1_x,price2_x,book_name_y,team_id_y,a_team_id_y,...,team_id_home,team_id_away,book_name,book_id_y,team_id,a_team_id,total1,total2,price1,price2
0,41100314,5Dimes,19,1610612759,1610612760,165.0,-175.0,5Dimes,1610612759,1610612760,...,1.610613e+09,1.610613e+09,Pinnacle Sports,238,1610612759,1610612760,201.5,201.5,-102.0,-108.0
20,41100304,5Dimes,19,1610612748,1610612738,-128.0,118.0,5Dimes,1610612748,1610612738,...,1.610613e+09,1.610613e+09,Pinnacle Sports,238,1610612748,1610612738,179.5,179.5,-112.0,102.0
40,41100315,5Dimes,19,1610612760,1610612759,190.0,-230.0,5Dimes,1610612760,1610612759,...,1.610613e+09,1.610613e+09,Pinnacle Sports,238,1610612760,1610612759,201.0,201.0,-103.0,-107.0
60,41100307,5Dimes,19,1610612738,1610612748,375.0,-420.0,5Dimes,1610612738,1610612748,...,1.610613e+09,1.610613e+09,Pinnacle Sports,238,1610612738,1610612748,178.5,178.5,-113.0,102.0
80,21200015,5Dimes,19,1610612754,1610612766,-280.0,255.0,5Dimes,1610612754,1610612766,...,NaN,NaN,Pinnacle Sports,238,1610612754,1610612766,183.0,183.0,-102.0,-108.0


In [927]:
#More cleaning
columns_to_drop = ['book_id_x', 'book_name_x', 
                   'book_id_y', 'book_name_y', 
                   'team_id_y', 'a_team_id_y',
                   'w' , 'l', 'w_pct', 'min', 'fgm',
                   'fga', 'fg_pct', 'fg3m', 'fg3a',
                   'fg3_pct', 'ftm', 'fta', 'ft_pct',
                   'oreb', 'dreb', 'ast', 'stl', 'blk',
                   'tov', 'pf', 'season_year', 
                   'season', 'reb', 'price1_y', 'price2_y',
                   'team_id', 'a_team_id', 'book_name', 'season_type',
                   'price1', 'price2']

#Map team names
games['Designated Team'] = games['matchup'].str[-3:]
abbrev_id_map = games[['Designated Team', 'game_team_id']] \
    .drop_duplicates() \
    .set_index('Designated Team')['game_team_id'].to_dict()

for book_name, book_df in merged_books.items():
    #Apply mapping to designated team and other team
    book_df['Designated Team'] = book_df['matchup'].str[-3:]
    book_df['game_team_id'] = book_df['Designated Team'].map(abbrev_id_map)
    book_df['Other Team'] = book_df['matchup'].str[0:3]
    book_df['game_team_id'] = book_df['Designated Team'].map(abbrev_id_map)
    book_df = book_df.copy()
    
    #Align the correct odds with each team
    #Betting datasets are weird since they use one price for home and other for away
    #but the "designated team" has the correct 'wl' value and it can be home or away
    book_df.loc[:, 'Designated Team ML Odds'] = np.where(
        book_df['game_team_id'] == book_df['team_id_x'],
        book_df['price1_x'],
        np.where(
            book_df['game_team_id'] == book_df['a_team_id_x'],
            book_df['price2_x'],
            np.nan 
        )
    )
    book_df.loc[:, 'Other Team ML Odds'] = np.where(
        book_df['game_a_team_id'] == book_df['team_id_x'],
        book_df['price1_x'],
        np.where(
            book_df['game_a_team_id'] == book_df['a_team_id_x'],
            book_df['price2_x'],
            np.nan
        )
    )
    book_df.loc[:, 'Designated Team Spread'] = np.where(
        book_df['game_team_id'] == book_df['team_id_x'],
        book_df['spread1'],
        np.where(
            book_df['game_team_id'] == book_df['a_team_id_x'],
            book_df['spread2'],
            np.nan
        )
    )
    book_df.loc[:, 'Other Team Spread'] = np.where(
        book_df['game_a_team_id'] == book_df['team_id_x'],
        book_df['spread1'],
        np.where(
            book_df['game_a_team_id'] == book_df['a_team_id_x'],
            book_df['spread2'],
            np.nan
        )
    )
    book_df.loc[:, 'Designated Team Points'] = book_df['pts'] 
    book_df.loc[:, 'Other Team Points'] = np.where(
        book_df['game_a_team_id'] == book_df['team_id_home'],
        book_df['pts_home'],
        np.where(
            book_df['game_a_team_id'] == book_df['team_id_away'],
            book_df['pts_away'],
            np.nan
        )
    )

    #Drop extra columns
    book_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    book_df.drop(columns=['price1_x', 'price2_x', 'spread1', 'spread2', 
                          'total2', 'pts_home', 'pts_away', 'team_id_home', 
                          'team_id_away', 'pts'], inplace=True, errors='ignore')
    
    #Finally merge together
    merged_books[book_name] = book_df
    
#Another sanity check
print(merged_books['5Dimes'].columns)
merged_books['5Dimes'].head()

Index(['game_id', 'team_id_x', 'a_team_id_x', 'game_date', 'matchup',
       'game_team_id', 'is_home', 'wl', 'game_a_team_id', 'total1',
       'Designated Team', 'Other Team', 'Designated Team ML Odds',
       'Other Team ML Odds', 'Designated Team Spread', 'Other Team Spread',
       'Designated Team Points', 'Other Team Points'],
      dtype='object')


,game_id,team_id_x,a_team_id_x,game_date,matchup,game_team_id,is_home,wl,game_a_team_id,total1,Designated Team,Other Team,Designated Team ML Odds,Other Team ML Odds,Designated Team Spread,Other Team Spread,Designated Team Points,Other Team Points
0,41100314,1610612759,1610612760,2012-06-02,SAS @ OKC,1610612760,t,W,1610612759,201.5,OKC,SAS,-175.0,165.0,-3.5,3.5,109,103.0
20,41100304,1610612748,1610612738,2012-06-03,MIA @ BOS,1610612738,t,W,1610612748,179.5,BOS,MIA,118.0,-128.0,2.0,-2.0,93,91.0
40,41100315,1610612760,1610612759,2012-06-04,SAS vs. OKC,1610612760,f,W,1610612759,201.0,OKC,SAS,190.0,-230.0,5.0,-5.0,108,103.0
60,41100307,1610612738,1610612748,2012-06-09,MIA vs. BOS,1610612738,f,L,1610612748,178.5,BOS,MIA,375.0,-420.0,8.0,-8.0,88,101.0
80,21200015,1610612754,1610612766,2012-11-02,CHA vs. IND,1610612754,f,L,1610612766,183.0,IND,CHA,-280.0,255.0,-6.5,6.5,89,NaN


In [928]:
#Let's look at this dataset and see how far off the spread were for the losing heavily favored team
heavy_books = {}
for book_name, book_df in merged_books.items():
    book_copy = book_df.copy()
    book_copy = book_copy[book_copy['Designated Team ML Odds'] <= -300]
    heavy_books[book_name] = book_copy

for book_name, book_df in heavy_books.items():
    losses = book_df[book_df['wl'] == 'L']
    print(losses['Designated Team Spread'].mean())

-8.98356807511737
-8.686075949367089
-8.786937901498929
-8.388613861386139
-8.734265734265735
-8.852209944751381
-8.81132075471698
-8.93498817966903
-8.686075949367089
-8.76006711409396


In [929]:
#On average, the sportsbook was off by around 8-9 points for losses when the team was favored by more than -300
#How much did they lose by on average?
for book_name, book_df in heavy_books.items():
    book_df['diff'] = book_df['Designated Team Points'] - book_df['Other Team Points']
    losses = book_df[book_df['wl'] == 'L']
    print(losses['diff'].mean())

-7.828715365239295
-8.204419889502763
-7.8729792147806
-8.010752688172044
-8.283464566929133
-7.901785714285714
-7.956632653061225
-7.9440203562340965
-8.204419889502763
-8.338345864661655


In [930]:
#Teams favored by -300 usually lost by around 7-8 points
#What about for teams favored by -1000?
massive_books = {}

for book_name, book_df in merged_books.items():
    book_copy = book_df.copy()
    book_copy = book_copy[book_copy['Designated Team ML Odds'] <= -1000]
    massive_books[book_name] = book_copy

for book_name, book_df in massive_books.items():
    losses = book_df[book_df['wl'] == 'L']
    print(losses['Designated Team Spread'].mean())

-13.058823529411764
-13.525
-13.051282051282051
-13.785714285714286
-13.217391304347826
-13.160714285714286
-13.2
-13.387096774193548
-13.525
-13.043478260869565


In [931]:
#How much did they lose by on average?
for book_name, book_df in massive_books.items():
    book_df['diff'] = book_df['Designated Team Points'] - book_df['Other Team Points']
    losses = book_df[book_df['wl'] == 'L']
    print(losses['diff'].mean())

-6.852941176470588
-6.4
-6.205128205128205
-6.857142857142857
-5.869565217391305
-6.392857142857143
-6.371428571428571
-6.548387096774194
-6.4
-6.521739130434782


In [932]:
#Teams favored by around 13 points usually lose by around 6, or 2 possessions (if taking 2 threes)
#Let's look at the proportions of the heavy and massive books to see how many games are dropped by sportsbooks
heavy_books = {}
for book_name, book_df in merged_books.items():
    book_copy = book_df.copy()
    book_copy = book_copy[book_copy['Designated Team ML Odds'] <= -300]
    heavy_books[book_name] = book_copy

all_books_df = pd.concat(heavy_books.values(), ignore_index=True)
total_wl_counts = all_books_df['wl'].value_counts()
total_loss_proportion = total_wl_counts.get('L', 0) / total_wl_counts.sum()
print(total_loss_proportion)

all_books_df = pd.concat(massive_books.values(), ignore_index=True)
total_wl_counts = all_books_df['wl'].value_counts()
total_loss_proportion = total_wl_counts.get('L', 0) / total_wl_counts.sum()
print(total_loss_proportion)

0.18389497132565524
0.08107265357031494


In [933]:
#18% of heavy favored teams lost and only 8% of massively favored teams lost
#Now looking at totals
for book_name, book_df in merged_books.items():
    book_df['total score'] = book_df['Designated Team Points'] + book_df['Other Team Points']
    book_df['points_away_from_total'] = book_df['total score'] - book_df['total1']    
    book_df['points_away_from_total_abs'] = abs(book_df['points_away_from_total'])
    print(book_df['points_away_from_total_abs'].mean())

13.669531192861845
13.677493606138107
13.661713737924932
13.678018728437655
13.611170568561873
13.597277710675462
13.616906184401744
13.666691171065207
13.677493606138107
13.65858120868744


In [934]:
#Looks like the books are off by around 13 points
#Does the over or under hit more often?
for book_name, book_df in merged_books.items():
    book_df['over'] = (book_df['total score'] > book_df['total1'])

all_books_df2 = pd.concat(merged_books.values(), ignore_index=True)
all_books_df2 = all_books_df2.dropna(subset=['Other Team Points'])
all_books_df2['over'].value_counts()

over
False    58054
True     55484
Name: count, dtype: int64

In [935]:
#Seems to be about the same
#Can Odds be a good predictor of a win or loss?
#Build Decision Tree Classifier to Predict Winners based on Odds and Spread only for each book
for book_name, book_df in merged_books.items():
    X = book_df[['Designated Team ML Odds', 'Designated Team Spread', 'Other Team ML Odds', 'Other Team Spread']]
    y = book_df['wl']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=3)
    tree = DecisionTreeClassifier(random_state=1)
    tree.fit(X_train, y_train)
    classes = tree.classes_
    print(book_name)
    classificationSummary(y_valid, tree.predict(X_valid), class_names=tree.classes_)

5Dimes
Confusion Matrix (Accuracy 0.6525)

       Prediction
Actual    L    W
     L 2017 1006
     W 1064 1869
BetOnline
Confusion Matrix (Accuracy 0.6582)

       Prediction
Actual    L    W
     L 1838  798
     W  969 1565
Bookmaker
Confusion Matrix (Accuracy 0.6436)

       Prediction
Actual    L    W
     L 2016  946
     W 1161 1789
Bovada
Confusion Matrix (Accuracy 0.6537)

       Prediction
Actual    L    W
     L 1542  668
     W  857 1337
Heritage
Confusion Matrix (Accuracy 0.6345)

       Prediction
Actual    L    W
     L 1202  575
     W  696 1004
Intertops
Confusion Matrix (Accuracy 0.6621)

       Prediction
Actual    L    W
     L 1672  761
     W  845 1475
JustBet
Confusion Matrix (Accuracy 0.6379)

       Prediction
Actual    L    W
     L 1833  922
     W 1048 1637
Pinnacle Sports
Confusion Matrix (Accuracy 0.6384)

       Prediction
Actual    L    W
     L 2087  891
     W 1252 1697
Sportsbetting
Confusion Matrix (Accuracy 0.6458)

       Prediction
Actual    L    

In [936]:
#Very interesting results above, seems like the tree only predicts the correct outcome about two thirds of the time
#What about for heavily favored teams?
for book_name, book_df in heavy_books.items():
    X = book_df[['Designated Team ML Odds', 'Designated Team Spread', 'Other Team ML Odds', 'Other Team Spread']]
    y = book_df['wl']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=3)
    tree = DecisionTreeClassifier(random_state=1)
    tree.fit(X_train, y_train)
    classes = tree.classes_
    print(book_name)
    classificationSummary(y_valid, tree.predict(X_valid), class_names=tree.classes_)

5Dimes
Confusion Matrix (Accuracy 0.7685)

       Prediction
Actual   L   W
     L  11 160
     W  68 746
BetOnline
Confusion Matrix (Accuracy 0.7663)

       Prediction
Actual   L   W
     L  13 147
     W  50 633
Bookmaker
Confusion Matrix (Accuracy 0.7486)

       Prediction
Actual   L   W
     L  30 157
     W 105 750
Bovada
Confusion Matrix (Accuracy 0.7793)

       Prediction
Actual   L   W
     L   7 140
     W  26 579
Heritage
Confusion Matrix (Accuracy 0.7188)

       Prediction
Actual   L   W
     L  21  95
     W  74 411
Intertops
Confusion Matrix (Accuracy 0.7851)

       Prediction
Actual   L   W
     L   6 130
     W  43 626
JustBet
Confusion Matrix (Accuracy 0.7609)

       Prediction
Actual   L   W
     L  19 143
     W  87 713
Pinnacle Sports
Confusion Matrix (Accuracy 0.7467)

       Prediction
Actual   L   W
     L  25 142
     W 105 703
Sportsbetting
Confusion Matrix (Accuracy 0.7604)

       Prediction
Actual   L   W
     L  16 136
     W  66 625
YouWager
Confusion

In [937]:
#Seems more reliable but not the best
#Let's try teams favored by -1000
for book_name, book_df in massive_books.items():
    X = book_df[['Designated Team ML Odds', 'Designated Team Spread', 'Other Team ML Odds', 'Other Team Spread']]
    y = book_df['wl']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=3)
    tree = DecisionTreeClassifier(random_state=1)
    tree.fit(X_train, y_train)
    classes = tree.classes_
    print(book_name)
    classificationSummary(y_valid, tree.predict(X_valid), class_names=tree.classes_)

5Dimes
Confusion Matrix (Accuracy 0.8441)

       Prediction
Actual   L   W
     L   2  12
     W  17 155
BetOnline
Confusion Matrix (Accuracy 0.8523)

       Prediction
Actual  L  W
     L  1  8
     W  5 74
Bookmaker
Confusion Matrix (Accuracy 0.8738)

       Prediction
Actual   L   W
     L   0  19
     W   7 180
Bovada
Confusion Matrix (Accuracy 0.8077)

       Prediction
Actual  L  W
     L  0  4
     W  1 21
Heritage
Confusion Matrix (Accuracy 0.7981)

       Prediction
Actual  L  W
     L  1 10
     W 11 82
Intertops
Confusion Matrix (Accuracy 0.8837)

       Prediction
Actual   L   W
     L   0   9
     W   6 114
JustBet
Confusion Matrix (Accuracy 0.8947)

       Prediction
Actual   L   W
     L   1  14
     W   6 169
Pinnacle Sports
Confusion Matrix (Accuracy 0.9056)

       Prediction
Actual   L   W
     L   0  12
     W   5 163
Sportsbetting
Confusion Matrix (Accuracy 0.8523)

       Prediction
Actual  L  W
     L  1  8
     W  5 74
YouWager
Confusion Matrix (Accuracy 0.8222

In [938]:
#Not perfect but accuracy increases more and more as a team is more favored
#Now I'm creating a logistic regression model for each book to compare to a team strengths model to see which one performs better
#I'm training the data on the entire set so we can get a prediction for every matchup based on the odds
import statsmodels.api as sm

for book_name, book_df in merged_books.items():
    #Use only odds to predict
    X = book_df[['Designated Team ML Odds', 'Designated Team Spread',
                 'Other Team ML Odds', 'Other Team Spread']]
    y = book_df['wl']
    y = y.map({'W': 1, 'L': 0})
    model_df = X.copy()
    model_df['y'] = y

    #No null values
    model_df = model_df.replace([np.inf, -np.inf], np.nan).dropna()
    X_clean = model_df.drop(columns='y')
    y_clean = model_df['y']

    X_const = sm.add_constant(X_clean)
    model = sm.GLM(y_clean, X_const, family=sm.families.Binomial()).fit()
    print(book_name)
    print(model.summary())

    # Predict win probabilities
    probs = model.predict(X_const)
    book_df.loc[X_clean.index, 'predicted_win_prob_bets'] = probs
    merged_books[book_name] = book_df

merged_books['5Dimes'].head()

5Dimes
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                14185
Model:                            GLM   Df Residuals:                    14180
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8267.8
Date:                Thu, 24 Apr 2025   Deviance:                       16536.
Time:                        11:36:55   Pearson chi2:                 1.41e+04
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1979
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const              

,game_id,team_id_x,a_team_id_x,game_date,matchup,game_team_id,is_home,wl,game_a_team_id,total1,...,Other Team ML Odds,Designated Team Spread,Other Team Spread,Designated Team Points,Other Team Points,total score,points_away_from_total,points_away_from_total_abs,over,predicted_win_prob_bets
0,41100314,1610612759,1610612760,2012-06-02,SAS @ OKC,1610612760,t,W,1610612759,201.5,...,165.0,-3.5,3.5,109,103.0,212.0,10.5,10.5,True,0.626792
20,41100304,1610612748,1610612738,2012-06-03,MIA @ BOS,1610612738,t,W,1610612748,179.5,...,-128.0,2.0,-2.0,93,91.0,184.0,4.5,4.5,True,0.425712
40,41100315,1610612760,1610612759,2012-06-04,SAS vs. OKC,1610612760,f,W,1610612759,201.0,...,-230.0,5.0,-5.0,108,103.0,211.0,10.0,10.0,True,0.332969
60,41100307,1610612738,1610612748,2012-06-09,MIA vs. BOS,1610612738,f,L,1610612748,178.5,...,-420.0,8.0,-8.0,88,101.0,189.0,10.5,10.5,True,0.238619
80,21200015,1610612754,1610612766,2012-11-02,CHA vs. IND,1610612754,f,L,1610612766,183.0,...,255.0,-6.5,6.5,89,NaN,NaN,NaN,NaN,False,0.715229


In [939]:
#Now create the X and W functions for the team strengths models
def create_X(df: pd.DataFrame) -> np.ndarray:
  # Dimensions of X
  n = len(df)
  p = df['Designated Team'].nunique()

  # Initialize with 0s
  X = np.zeros((n,p))

  # Sort teams
  team_sort = df['Designated Team'].sort_values().unique()

  # Assign 1s and -1s accordingly
  for i in range(len(X)):

    # Obtain team names for game i
    home_team = df['Designated Team'].iloc[i]
    away_team = df['Other Team'].iloc[i]

    # Obtain corresponding indices
    home_idx = np.where(team_sort==home_team)[0][0]
    away_idx = np.where(team_sort==away_team)[0][0]

    # Assign values
    X[i,home_idx] = 1
    X[i,away_idx] = -1

  return X

In [940]:
#Not regarding home team advantage since it would be lots more preprocessing due to the nature of the betting datasets
def create_W(n_teams: int, constraint_type: int) -> np.ndarray:

  #Start with Identity Matrix
  W = np.identity(n_teams)

  #Assign -1 to all values in last row
  if constraint_type==1:
    W[-1,:] = 0
  elif constraint_type==2:
    W[-1,:] = -1
  else:
    raise ValueError("Parameter 'constraint_type' only takes values 1 or 2.")

  #Remove last team's column
  W = W[:,:-1]

  return W

In [941]:
#Now apply everything to end up with prediction probabilities for each matchup
#Also using contraint type 2 to add wizards back in
for book_name, book_df in merged_books.items():
    print(book_name)

    #Only keep rows with valid win/loss outcomes
    book_df = book_df[book_df['wl'].isin(['W', 'L'])].reset_index(drop=True)

    #Create X and y
    X = create_X(book_df)
    y = book_df['wl'].map({'W': 1, 'L': 0}).values

    #Create constraint matrix W
    W = create_W(n_teams=X.shape[1], constraint_type=2)
    logreg = sm.GLM(endog=y, exog=X @ W, family=sm.families.Binomial()).fit()
    teams = book_df['Designated Team'].sort_values().unique()
    xnames = list(teams[:-1])        
    print(logreg.summary(xname=xnames))

    #Team strengths
    theta = logreg.params
    theta_all = np.append(theta, -1*theta.sum())
    book_team_strengths[book_name] = theta_all
    df_ests = pd.DataFrame(columns=['Team', 'Strengths'])
    df_ests['Team'] = teams
    df_ests['Strengths'] = theta_all
    print(df_ests)

    #Predict
    y_book = np.array(book_df['wl']).reshape((-1,1))
    y_book.shape
    coefs = theta_all.reshape((-1, 1))
    X = create_X(df=book_df)
    y_pred = X @ coefs
    y_pred_probs = 1/(1+np.exp(-y_pred))
    book_df['predicted_win_prob_strengths'] = y_pred_probs
    merged_books[book_name] = book_df


5Dimes
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                14890
Model:                            GLM   Df Residuals:                    14858
Model Family:                Binomial   Df Model:                           31
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -9929.7
Date:                Thu, 24 Apr 2025   Deviance:                       19859.
Time:                        11:36:56   Pearson chi2:                 1.49e+04
No. Iterations:                     4   Pseudo R-squ. (CS):            0.05106
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ATL            0.0445      0.062      0.718  

In [942]:
#Now assess the accuracies of both models to determine the better one
from sklearn.metrics import log_loss, brier_score_loss, roc_auc_score, accuracy_score

for book_name, book_df in merged_books.items():
    book_df = book_df.dropna(subset=['predicted_win_prob_strengths', 'predicted_win_prob_bets', 'wl'])
    y_true = book_df['wl'].values
    y_true = np.where(y_true == 'W', 1, 0)    
    pred_strengths = book_df['predicted_win_prob_strengths'].values
    pred_bets = book_df['predicted_win_prob_bets'].values

    metrics = {
        "Log Loss": (
            log_loss(y_true, pred_strengths),
            log_loss(y_true, pred_bets)
        ),
        "Brier Score": (
            brier_score_loss(y_true, pred_strengths),
            brier_score_loss(y_true, pred_bets)
        ),
        "AUC": (
            roc_auc_score(y_true, pred_strengths),
            roc_auc_score(y_true, pred_bets)
        ),
        "Accuracy (threshold 0.5)": (
            accuracy_score(y_true, pred_strengths > 0.5),
            accuracy_score(y_true, pred_bets > 0.5)
        )
    }

print("Metric Comparison (Strengths vs Bets):")
for metric, (val_strengths, val_bets) in metrics.items():
    print(f"{metric}: Strengths={val_strengths:.4f}, Bets={val_bets:.4f}")

Metric Comparison (Strengths vs Bets):
Log Loss: Strengths=0.6587, Bets=0.5920
Brier Score: Strengths=0.2333, Bets=0.2034
AUC: Strengths=0.6481, Bets=0.7498
Accuracy (threshold 0.5): Strengths=0.6067, Bets=0.6834


In [943]:
#All in all it looks like the betting model is slightly better than the team strengths model, beating it in every metric
#Therefore, the sportsbooks know what they're doing when it comes to setting odds, but there are not perfect in the slightest
#Conclusion: odds are not always right but are decent predictors and better than some model staples like team strengths